# Databricks RAG Demo Notebook
---------------------------------------
This notebook demonstrates:
1. Vector search against a product index
2. Retrieval of product name + description
3. Construction of a RAG prompt
4. Sending prompt to an LLM endpoint

### Setup

In [0]:
%python
%pip install databricks-sdk
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
%python
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole
import mlflow
import json

# Initialize client
w = WorkspaceClient()
vsc = VectorSearchClient(disable_notice=True)

# Get the index
index = vsc.get_index(index_name="workspace.default.demo_product_texts_vsi")

In [0]:
%python
# COMMAND 2: Define retrieval function
def retrieve_products(query: str, k: int = 3):
    """Perform a vector similarity search against the index."""
    # Run a similarity search using text
    results = index.similarity_search(
        query_text = "comfortable shoes for walking long distances",
        columns = ["id","name", "text"],   # only return these columns
        num_results = 5,
        disable_notice=True
    )
    return results["result"]["data_array"]

retrieve_products_test = retrieve_products("What shoes are best for rocky trails?",3)
print(retrieve_products_test)

In [0]:
%python
# COMMAND 3: Format retrieved context for RAG
def format_context(rows):
    formatted = []
    for r in rows:
        formatted.append(f"""
            Product ID: {r[0]}
            Name: {r[1]}
            Description: {r[2]}
            """)
    return "\n".join(formatted)

format_context_test = format_context(retrieve_products_test)
print(format_context_test)

In [0]:
%python
# COMMAND 4: Build RAG prompt
def build_prompt(query: str, context: str):
    system_prompt = """
You are a helpful product expert. You answer customer questions using only the information provided in the retrieved product context.
If the context does not contain the answer, say: "The catalog does not include that information."
Do not make up product details, names, or features.
"""

    user_prompt = f"""
User question:
{query}

Here are the retrieved products from the catalog:
{context}

Using ONLY this information, provide the best possible answer.
"""

    return system_prompt, user_prompt

query = "comfortable shoes for walking long distances"
build_prompt_test = build_prompt(query, format_context_test)
system_prompt_test = build_prompt_test[0]
user_prompt_test = build_prompt_test[1]

print(system_prompt_test)
print(user_prompt_test)

In [0]:
%python
# COMMAND 5: Query LLM endpoint
# Replace with your Databricks model serving endpoint
MODEL_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"

def run_llm(system_prompt, user_prompt):
    # client = mlflow.deployments.get_deploy_client("databricks")
    messages = [
        ChatMessage(
            role=ChatMessageRole.SYSTEM,
            content=system_prompt
        ),
        ChatMessage(
            role=ChatMessageRole.USER,
            content=user_prompt
        )
    ]
    response = w.serving_endpoints.query(
        name=MODEL_ENDPOINT,
        messages=messages,
        max_tokens=500,
        temperature=0.7

    )
    return response.choices[0].message.content

run_llm_test = run_llm(system_prompt_test, user_prompt_test)
print(run_llm_test)

In [0]:
%python
# COMMAND 6: Full RAG pipeline wrapper
def rag_answer(query: str):
    rows = retrieve_products(query)
    context = format_context(rows)
    system_prompt, user_prompt = build_prompt(query, context)
    answer = run_llm(system_prompt, user_prompt)
    return answer, context

In [0]:
%python
# COMMAND 7: Example usage
query = "What are the most comfortable shoes?"
answer, retrieved_context = rag_answer(query)

# print("Retrieved Context:\n", retrieved_context)
print("\nRAG Answer:\n", answer)